In [1]:
import xlrd
import numpy as np

#read class
def readTranscript(filename='genes_transcript.xlsx'):
    sheet = xlrd.open_workbook(filename).sheet_by_index(0)
    nRows = sheet.nrows
    my_class_label = []
    my_class_value = []
    for rowNum in range(0,nRows):
        rowList = []
        row = sheet.row_values(rowNum)
        rowLabel = row.pop(0)
        my_class_label.append([rowLabel])
        for el in row:
            rowList.append([el])
        rowList = np.array(rowList)
        my_class_value.append(rowList)
    my_class_label = np.array(my_class_label)
    my_class_value = np.array(my_class_value)
    #print(my_class_label)
    #print(my_class_value)
    return my_class_label, my_class_value

#read data
def readData(filename='gene_data.xlsx'):
    sheet = xlrd.open_workbook(filename).sheet_by_index(0)
    nRows = sheet.nrows
    my_data_value = []
    for rowNum in range(0,nRows):
        rowList = []
        row = sheet.row_values(rowNum)
        for el in row:
            rowList.append([el])
        rowList = np.array(rowList)
        my_data_value.append(rowList)
    my_data_value = np.array(my_data_value)
    #print(my_data_value)
    return my_data_value

class_name, class_value = readTranscript()
data_value = readData()

In [2]:
#Chi square feature selection

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

#select 100 features
def chisq_feature_selection(X, y,numoffeature=100):
    
    #chi square
    chisq = SelectKBest(chi2, k=numoffeature)
    Xchi = chisq.fit_transform(X, y)
    return Xchi


In [3]:
data = np.copy(data_value)
#print(data)

In [4]:
#remove header(name of each sample)
data = np.delete(data, 0,axis=0)
#print(data1)

#remove first row(ID row)
data = np.delete(data, 0, axis=1)
#print(data2)

#turn 3dd list to 2d list
#print(data2.shape)
data = np.reshape(data, (data.shape[0], data.shape[1],))
#print(data3.shape)

#data to match the class
data = data.transpose()
#print(data4)

#change from str to float
data_float = data.astype(np.float)
#print(data_float)

In [5]:
#select gleason (total) score
y = class_value[3]
#print(y)

#change from str to float
y_float = y.astype(np.float)
#print(y_float)

#reshape y to 1d
y_float = np.reshape(y_float, (y_float.shape[0]))


In [6]:
X = chisq_feature_selection(data_float, y_float)
print(X.shape)

(494, 100)


In [7]:
#train test split
#from sklearn.model_selection import train_test_split
#train_data, test_data, train_class, test_class = train_test_split(X, y_float, test_size=0.1, random_state=20201217)

print(y_float.shape)

(494,)


In [8]:
#max feature selected/ calculated
max_feature = 600

In [9]:
#calculate chi square and sort the column according to the scores
selector = SelectKBest(chi2, k=max_feature).fit(data_float, y_float)
x_new = selector.transform(data_float)
selector_scores = selector.scores_
#print(selector_scores)
#print(x_new.shape)
Z = [elx for _,elx in sorted(zip(selector_scores,x_new))]

Zchi = np.array(Z)
#print(Zchi)

In [ ]:
#cross val with chi square
import warnings
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.model_selection import cross_val_score

svmClassifier = svm.SVC(kernel='poly', degree=5)

#disable warning
warnings.filterwarnings(action='ignore')

chisq_scoreList = []
for fea in range(1, max_feature+1):
    Xfea = Zchi[:,:fea]
    
    scores = cross_val_score(svmClassifier, Xfea, y_float, cv=10)
    chisq_scoreList.append(scores[-1])


In [ ]:
#plot result
#print(chisq_scoreList)
plt.figure()
plt.title('cross val score under different number of features')
plt.plot(range(1,max_feature+1),chisq_scoreList )
plt.show()

In [ ]:
#mutual information feature selection
from sklearn.feature_selection import mutual_info_classif

selector = SelectKBest(mutual_info_classif, k=max_feature).fit(data_float, y_float)
x_new = selector.transform(data_float)

selector_scores = selector.scores_
#print(selector_scores)

In [ ]:
Z = []
for x_row in x_new:
    z = [elx for _,elx in sorted(zip(selector_scores, x_row))]
    Z.append(np.array(z))
ZmutalInfo = np.array(Z)
#print(ZmutalInfo)

In [ ]:
#cross val with mutual info
import warnings
from matplotlib import pyplot as plt

svmClassifier = svm.SVC(kernel='poly', degree=5)

#disable warning
warnings.filterwarnings(action='ignore')

mutualInfo_scoreList = []
for fea in range(1, max_feature+1):
    Xfea = ZmutalInfo[:,:fea]
    
    scores = cross_val_score(svmClassifier, X, y_float, cv=10)
    mutualInfo_scoreList.append(scores[-1])

In [ ]:
#plot result
#print(mutualInfo_scoreList)
plt.figure()
plt.title('cross val score under different number of features')
plt.plot(range(1,max_feature+1),mutualInfo_scoreList )
plt.show()

In [ ]:
#f test feature selection
from sklearn.feature_selection import f_classif

selector = SelectKBest(mutual_info_classif, k=max_feature).fit(data_float, y_float)
x_new = selector.transform(data_float)
selector_scores = selector.scores_
#print(selector_scores)

In [ ]:
Z = []
for x_row in x_new:
    z = [elx for _,elx in sorted(zip(selector_scores, x_row))]
    Z.append(np.array(z))

Zftest = np.array(Z)
#print(Zftest)

In [ ]:
#cross val with f test
import warnings
from matplotlib import pyplot as plt

svmClassifier = svm.SVC(kernel='poly', degree=5)

#disable warning
warnings.filterwarnings(action='ignore')

f_test_scoreList = []
for fea in range(1, max_feature+1):
    Xfea = Zftest[:,:fea]
    
    scores = cross_val_score(svmClassifier, X, y_float, cv=10)
    f_test_scoreList.append(scores[-1])

In [ ]:
#plot result
#print(f_test_scoreList)
plt.figure()
plt.title('cross val score under different number of features')
plt.plot(range(1,max_feature+1),f_test_scoreList )
plt.show()

In [ ]:
plt.figure()
plt.title('cross val score under different number of features')
plt.plot(range(1,max_feature+1),chisq_scoreList, label='chi square' )
plt.plot(range(1,max_feature+1),mutualInfo_scoreList, label='mutual info' )
plt.plot(range(1,max_feature+1),f_test_scoreList, label='f test' )
plt.legend()
plt.show()